# 3. Hierarchical Bayesian Logistic Regression (HLR)

This section implements a hierarchical extension of the Bayesian logistic regression model on the German credit dataset. We introduce two-way interaction terms to expand our feature space to 300 dimensions. Additionally, the prior variance $\sigma^2$ is now treated as an unknown parameter and given an exponential prior.

To keep the parameter space unconstrained for HMC/NUTS, we sample $v = \log(\sigma^2)$ instead of $\sigma^2$. The log-posterior is adjusted with the Jacobian of this transformation.

In [1]:
import numpy as np
import pandas as pd
from scipy.special import expit
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot as plt

from bml.samplers import nuts, hmc
from bml import metrics

In [2]:
# Fetch the German Credit (numeric) dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data-numeric"
data = pd.read_csv(url, sep=r'\s+', header=None)

X_raw = data.iloc[:, :-1].values
y_raw = data.iloc[:, -1].values

# Map y from {1, 2} to {1, -1}
y = np.where(y_raw == 1, 1, -1)

# Normalize original features to zero mean and unit variance
X_norm = (X_raw - np.mean(X_raw, axis=0)) / np.std(X_raw, axis=0)

# Expand predictors to include 2-way interactions (24 -> 300 dimensions)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_expanded = poly.fit_transform(X_norm)

# Prepend a column of ones for the intercept alpha
X = np.hstack([np.ones((X_expanded.shape[0], 1)), X_expanded])
N, d_coeffs = X.shape  # N=1000, d_coeffs=301 (alpha + 300 beta coefficients)

# Total parameters d = 301 coefficients + 1 variance parameter
d = d_coeffs + 1

In [ ]:
lam = 0.01  # Rate parameter for the exponential prior on sigma^2

def log_p(theta):
    """
    Evaluates the log-posterior for Hierarchical Bayesian Logistic Regression.
    theta[:301] = alpha and beta coefficients
    theta[301] = v = log(sigma^2)
    """
    if np.any(np.abs(theta) > 500.0) or theta[-1] < -20.0 or theta[-1] > 20.0:
        return -np.inf
    
    coeffs = theta[:d_coeffs]
    v = theta[-1]

    sigma_sq = np.exp(v)
    inv_sigma_sq = np.exp(-v)  # 1/sigma^2 for numerical stability

    # Log-likelihood
    z = -y * np.dot(X, coeffs)
    log_likelihood = -np.sum(np.logaddexp(0, z))
    
    # Log-prior
    log_prior = -0.5 * (np.sum(coeffs**2) * inv_sigma_sq) - (N / 2) * v - lam * sigma_sq + v
    
    return log_likelihood + log_prior

def grad_log_p(theta):
    """
    Evaluates the gradient of the log-posterior for HLR.
    """
    theta_safe = np.clip(theta, -1000.0, 1000.0)
    theta_safe[-1] = np.clip(theta_safe[-1], -20.0, 20.0)
    
    coeffs = theta_safe[:d_coeffs]
    v = theta_safe[-1]

    sigma_sq = np.exp(v)
    inv_sigma_sq = np.exp(-v)  # 1/sigma^2 for numerical stability
    grad = np.zeros_like(theta)
    
    # Gradient
    z = -y * np.dot(X, coeffs)
    s = expit(z)
    grad_likelihood_coeffs = np.dot(X.T, y * s)
    grad_prior_coeffs = -coeffs * inv_sigma_sq
    
    grad[:d_coeffs] = grad_likelihood_coeffs + grad_prior_coeffs
    
    # Gradient w.r.t. v = log(sigma^2)
    grad[-1] = 0.5 * np.sum(coeffs**2) * inv_sigma_sq - (N / 2.0) - lam * sigma_sq + 1.0
    
    return grad

In [4]:
# Initial point
theta0 = np.zeros(d)
theta0[-1] = np.log(100.0) 

M = 2000             # Total iterations
M_adapt = 1000       # Warmup/adaptation iterations
delta = 0.65         # Target acceptance rate

results_hlr = {}

for sampler_name, SamplerClass in [("Dual Averaging NUTS", nuts.DualAveragingNUTS), 
                                   ("Dual Averaging HMC", hmc.DualAveragingHMC)]:
    print(f"Starting {sampler_name} sampling for Hierarchical LR...")
    
    sampler = SamplerClass(L=log_p, grad=grad_log_p)
    
    if sampler_name == "Dual Averaging HMC":
        # The optimal lambda length for HLR in the paper is roughly 0.14
        samples = sampler.sample(theta0, delta=delta, lam=0.14, M=M, M_adapt=M_adapt)
    else:
        samples = sampler.sample(theta0, delta=delta, M=M, M_adapt=M_adapt)
        
    print(f"Finished sampling with {sampler_name}. Output shape: {samples.shape}")
    
    valid_samples = samples[M_adapt:]
    
    # Calculate worst-case ESS across all 302 dimensions
    sample_means = np.mean(valid_samples, axis=0)
    sample_vars = np.var(valid_samples, axis=0)
    
    min_ess = float('inf')
    for dim in range(d):
        dim_samples = valid_samples[:, dim]
        mu = sample_means[dim]
        var = sample_vars[dim]
        
        ess_mean = metrics.compute_ess_1d(dim_samples, mu, var)
        
        moment_samples = (dim_samples - mu)**2
        moment_mu = np.mean(moment_samples)
        moment_var = np.var(moment_samples)
        ess_variance = metrics.compute_ess_1d(moment_samples, moment_mu, moment_var)
        
        min_ess = min(min_ess, ess_mean, ess_variance)
        
    results_hlr[sampler_name] = min_ess
    print(f"Worst-case ESS across all dimensions for {sampler_name}: {min_ess:.4f}\n")

Starting Dual Averaging NUTS sampling for Hierarchical LR...
Finished sampling with Dual Averaging NUTS. Output shape: (2001, 302)
Worst-case ESS across all dimensions for Dual Averaging NUTS: 3.1770

Starting Dual Averaging HMC sampling for Hierarchical LR...


KeyboardInterrupt: 